# Beat the Books

#### A data science project by Jonathan Sears

### Project Plan
The main goal of this project is to find a way to profit of of sports betting. There are several reasons I want to do this. First I want to make money, I feel like that is pretty self explanatory. Additionally, sports books and casinos always stack the odds in their favor, so I think it would be cool to find a way to beat them at their own game. 

### The Plan
My approach to beat the books will be as follows:

1) Build a couple different machine learning models. One that given some data we can aquire before a game starts, predicts the winner of a game (moneyline), one that predicts the point differential (spread), and one that predicts the total number of points (over under). 
2) Scrape the odds from many different sportsbooks. Since sportsbooks operate independently, their odds are constantly changing. Exploting these discrepencies to get the best odds for any bet is essential if you want to beat the books. 
3) Using some math I'm going to calculate my predicted expected value for a given bet, if the expected value > 0, then I will classify it as a winning bet. Theoretically, if I place enough bets and my model is accurate, I should be able to beat the books. 

# Math

### Expected Value and the Law of Large Numbers
There is some important math we will need to have a strong understanding of during this project. The plus EV strategy relies on identifying bets with positive expected values, so it's important to have a strong understanding of what expected values are. The expected value of a bet will be:

    EV = P(event A happens)* (Payoff of event A) + P(event B) * (Payoff of event B)

When looking at this we can see how important it is to properly asses the probability of a given event happening is, as that's what will be the real difference between making and losing money.

The second important tool is the law of large numbers, which states that the more bets we place, the higher probability we have of reaching our expected value. This is important because while we might have a positive EV on a bet this doesn't guarantee that it will hit. What the law of large numbers is saying that if we place 1000 bets with 50% probability of hitting, then we have an incredibly high probability of hitting around 500 of them. Basically, the more bets we place, the closer we should get to converging to our expected value.  

### The Kelly Criterion
The Kelly Criterion is a simple mathematical formula we can use to size our bets:

f<sup>*</sup> = p - (1-p)/b 


**f<sup>*</sup>** is the fraction of our bankroll we should put on the bet

**p** is our estimated probability of winning

**b** is the proportion of the bet we stand to win (eg for 2:1 odds b =2)

# Data
The two datasets I found that I think will come in handy for this project are the 538 ELO model dataset and the spreadspoke historical odds dataset. Unfortunately I can't use the 538 ELO model as a predictor when building my own model, as it was discontinued before the 2023 NFL season. However, I think it could still be useful to compare it to my own model to get a gaugue of how well I'm doing. The spreadspoke dataset will likely be one of the most important finds for this whole project as historical odds, spreads, and over under lines will come in extremely handy when building my own model. Finding free data about the NFL turned out to be a lot harder than I thought. I couldn't find any datasets that had historical box scores or anything like that, so instead I decided to make my own by using webscraping. Lastly I needed to find a way to get current odds of NFL games from a wide range of sportsbooks. The Odds API came in extremely handy for this

**Data Sources:**
 
538 NFL ELO: https://github.com/fivethirtyeight/data/tree/master/nfl-elo

Spreadspoke: https://www.kaggle.com/datasets/tobycrabtree/nfl-scores-and-betting-data

Box scores scraped from: https://www.footballdb.com/games/

Odds API: https://the-odds-api.com/ 



Other potential sources:
@misc{gilani_sdvpy_2021,
  author = {Gilani, Saiem},
  title = {sportsdataverse-py: The SportsDataverse's Python Package for Sports Data.},
  url = {https://py.sportsdataverse.org},
  season = {2021}
}

NFL Data: https://pypi.org/project/nfl-data-py/

PFR webscraper: https://pypi.org/project/pro-football-reference-web-scraper/ 

League Average data by season from PFR: https://www.pro-football-reference.com/years/NFL/index.htm 

# ETL

In [1]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
from bs4 import BeautifulSoup
import re 
import json
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error
from random import randint
import sportsdataverse.nfl as sdv_nfl
pd.set_option('display.max_columns',None)

In [2]:
#First lets read in our data
games = pd.read_csv('./Data/spreadspoke_scores.csv')
teams = pd.read_csv('./Data/nfl_teams.csv')
stadiums = pd.read_csv('./Data/nfl_stadiums.csv', encoding="unicode_escape")
past_elo = pd.read_csv('./Data/nfl_elo.csv')
recent_elo = pd.read_csv('./Data/nfl_elo_latest.csv')

In [3]:
#Let's also check our data types to make sure everything looks okay
games.dtypes
past_elo.dtypes
teams.dtypes
stadiums.dtypes

#lets change the neutral site from the elo stasets to a boolean type
past_elo['neutral'] = past_elo['neutral'].astype(bool)
recent_elo['neutral'] = recent_elo['neutral'].astype(bool)

#convert the date columns to datetime object
past_elo.loc[:,'date'] = pd.to_datetime(past_elo['date']).dt.floor('D')
recent_elo.loc[:,'date'] = pd.to_datetime(recent_elo['date']).dt.floor('D')
games.loc[:,'schedule_date'] = pd.to_datetime(games['schedule_date']).dt.floor('D')


In [4]:
# There's a lot of data in the elo datasets and a lot of it won't be useful for us. Lets drop all the data from before the superbowl era
sb_era_elo = past_elo[past_elo['season'] >= 1966]
#and lets add te recent games to this dataset as well
sb_era_elo = pd.concat([sb_era_elo,recent_elo])
#reindex the df
sb_era_elo = sb_era_elo.reindex(index=range(len(sb_era_elo)),copy=False)
#drop a random column that has all NaNs
sb_era_elo.dropna(subset = ['team1','team2','date','elo1_pre','elo2_pre'],inplace=True)

In [5]:
#Lets also drop games without betting data from the scores dataset
games.dropna(subset='spread_favorite', inplace=True)
games.dropna(subset='over_under_line', inplace=True)


Let's make some new columns indicating the winner of the game, who covered the spread, and if the over hit. 
For the over we will use 0 if the over did not hit, 1 if the over did hit, and 2 if the game was a push

In [6]:
def winner(df):
    if df['score_home'] > df['score_away']:
        return df['team_home']
    elif df['score_away'] > df['score_home']:
        return df['team_away']
    else:
        return 'Tie'
    
def over(df):
    if float(df['score_home'] + df['score_away']) > float(df['over_under_line']):
        return 1
    elif float(df['score_home'] + df['score_away']) < float(df['over_under_line']):
        return 0
    else:
        return 2


games['winner'] = games.apply(winner, axis = 1)
games['over'] = games.apply(over, axis = 1)

### Merging Datasets!

Let's merge the games and elo datasets into one massive dataset we can use to build our model

In [7]:
#Define functions to get the abbreviation for each team in the games dataset
def find_home_team_abbrev(df,):
    match = teams[teams['team_name'] == df['team_home']]
    abrev = match.iloc[0]['team_id']
    return abrev
def find_away_team_abbrev(df,):
    match = teams[teams['team_name'] == df['team_away']]
    abrev = match.iloc[0]['team_id']
    return abrev
games['home_abrev'] = games.apply(find_home_team_abbrev,axis = 1)
games['away_abrev'] = games.apply(find_away_team_abbrev,axis = 1)



In [8]:
#Create a gameID from the two date a dame was played, and the two team name abbreviations in alphabetical order
#function made for the teams_df
def make_game_id(teams_df):
    team1 = teams_df['home_abrev']
    team2 = teams_df['away_abrev']
    teams = [team1, team2]
    sorted_teams = sorted(teams)
    date_str = str(teams_df['schedule_date'])
    gameID = date_str + ' ' + sorted_teams[0] + ' vs ' + sorted_teams[1]
    return gameID
games['gameID'] = games.apply(make_game_id, axis = 1)
#function made for the elo_df
def make_game_id_2(elo_df):
    team1 = elo_df['team1']
    team2 = elo_df['team2']
    teams = [team1, team2]
    sorted_teams = sorted(teams)
    date_str = str(elo_df['date'])
    if type(teams[0]) != str:
        print(teams[0])
    if type(teams[1]) != str:
        print(teams[1])
    gameID = date_str + ' ' + sorted_teams[0] + ' vs ' + sorted_teams[1]
    return gameID
sb_era_elo['gameID'] = sb_era_elo.apply(make_game_id_2, axis = 1)

Now, we merge!

In [9]:
#merge the datasets
master_df = sb_era_elo.merge(games,on=['gameID'],how='inner')
master_df

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,qbelo1_pre,qbelo2_pre,qb1,qb2,qb1_value_pre,qb2_value_pre,qb1_adj,qb2_adj,qbelo_prob1,qbelo_prob2,qb1_game_value,qb2_game_value,qb1_value_post,qb2_value_post,qbelo1_post,qbelo2_post,score1,score2,quality,importance,total_rating,gameID,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,winner,over,home_abrev,away_abrev
0,2022-09-08 00:00:00,2022.0,False,NaN,LAR,BUF,1614.761436,1614.246762,0.593181,0.406819,1576.964901,1652.043296,1582.712236,1613.252889,Matthew Stafford,Josh Allen,200.283690,259.606335,5.938708,9.144750,0.534145,0.465855,-47.502436,402.149420,175.505078,273.860644,1549.330347,1646.634779,10.0,31.0,91.0,59.0,75.0,2022-09-08 00:00:00 BUF vs LAR,2022-09-08 00:00:00,2022,1,False,Los Angeles Rams,10.0,31.0,Buffalo Bills,BUF,-2.0,52.0,SoFi Stadium,False,72.0,0.0,NaN,indoor,Buffalo Bills,0,LAR,BUF
1,2022-09-11 00:00:00,2022.0,False,NaN,MIA,NE,1539.979483,1537.002981,0.596597,0.403403,1560.633403,1516.349061,1518.380774,1513.692959,Tua Tagovailoa,Mac Jones,139.176472,143.536642,-0.538674,0.149008,0.582123,0.417877,179.561802,122.148555,143.215005,141.397833,1539.874207,1492.199526,20.0,7.0,56.0,85.0,71.0,2022-09-11 00:00:00 MIA vs NE,2022-09-11 00:00:00,2022,1,False,Miami Dolphins,20.0,7.0,New England Patriots,MIA,-3.0,46.5,Hard Rock Stadium,False,NaN,NaN,NaN,NaN,Miami Dolphins,0,MIA,NE
2,2022-09-11 00:00:00,2022.0,False,NaN,CIN,PIT,1558.193285,1510.143510,0.657183,0.342817,1538.985268,1529.351527,1550.251126,1473.261200,Joe Burrow,Mitch Trubisky,173.650126,168.278069,3.742751,3.346756,0.675155,0.324845,23.319530,111.575433,158.617066,162.607806,1530.384153,1493.128173,20.0,23.0,56.0,75.0,66.0,2022-09-11 00:00:00 CIN vs PIT,2022-09-11 00:00:00,2022,1,False,Cincinnati Bengals,20.0,23.0,Pittsburgh Steelers,CIN,-7.0,44.5,Paycor Stadium,False,NaN,NaN,NaN,NaN,Pittsburgh Steelers,0,CIN,PIT
3,2022-09-11 00:00:00,2022.0,False,NaN,ATL,NO,1436.671709,1543.815038,0.439647,0.560353,1430.691471,1549.795276,1379.244628,1516.085009,Marcus Mariota,Jameis Winston,111.532470,151.069226,-12.435739,3.305974,0.357017,0.642983,314.391296,151.832530,131.818353,151.145557,1374.515041,1520.814596,26.0,27.0,19.0,54.0,37.0,2022-09-11 00:00:00 ATL vs NO,2022-09-11 00:00:00,2022,1,False,Atlanta Falcons,26.0,27.0,New Orleans Saints,NO,-5.5,44.0,Mercedes-Benz Stadium,False,72.0,0.0,NaN,indoor,New Orleans Saints,1,ATL,NO
4,2022-09-11 00:00:00,2022.0,False,NaN,NYJ,BAL,1364.983381,1508.988219,0.388223,0.611777,1344.202019,1529.769581,1391.094637,1546.006711,Joe Flacco,Lamar Jackson,110.242373,162.712934,19.852514,-3.618196,0.384031,0.615969,116.176803,112.207146,110.835816,157.662356,1370.565348,1566.536000,9.0,24.0,34.0,51.0,43.0,2022-09-11 00:00:00 BAL vs NYJ,2022-09-11 00:00:00,2022,1,False,New York Jets,9.0,24.0,Baltimore Ravens,BAL,-6.5,44.0,MetLife Stadium,False,NaN,NaN,NaN,NaN,Baltimore Ravens,0,NYJ,BAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6679,2009-12-27 00:00:00,2009.0,False,NaN,IND,NYJ,1757.422000,1526.277000,0.846154,0.153846,1704.465000,1579.234000,1729.831249,1524.128350,Peyton Manning,Mark Sanchez,230.162954,41.671763,9.160546,-23.545265,0.845426,0.154574,184.499836,7.703544,225.596642,38.274941,1676.888359,1577.071240,15.0,29.0,92.0,NaN,NaN,2009-12-27 00:00:00 IND vs NYJ,2009-12-27 00:00:00,2009,16,False,Indianapolis Colts,15.0,29.0,New York Jets,IND,-3.0,40.5,Lucas Oil Stadium,False,72.0,0.0,NaN,indoor,New York Jets,1,IND,NYJ
6680,2009-12-27 00:00:00,2009.0,False,NaN,PHI,DEN,1661.946000,1513.229000,0.773863,0.226137,1667.661000,1507.514000,1655.073103,1513.554908,Do

This dataframe is massive and has some duplicate data, lets clean it up a bit. First Let's drop the columns we don't need anymore. Since we are using the data to predict the outcome of the game, the only relavent data is the data before the game, so let's drop all the elo adjustments that happen after the game. 

In [10]:
master_df.columns

Index(['date', 'season', 'neutral', 'playoff', 'team1', 'team2', 'elo1_pre',
       'elo2_pre', 'elo_prob1', 'elo_prob2', 'elo1_post', 'elo2_post',
       'qbelo1_pre', 'qbelo2_pre', 'qb1', 'qb2', 'qb1_value_pre',
       'qb2_value_pre', 'qb1_adj', 'qb2_adj', 'qbelo_prob1', 'qbelo_prob2',
       'qb1_game_value', 'qb2_game_value', 'qb1_value_post', 'qb2_value_post',
       'qbelo1_post', 'qbelo2_post', 'score1', 'score2', 'quality',
       'importance', 'total_rating', 'gameID', 'schedule_date',
       'schedule_season', 'schedule_week', 'schedule_playoff', 'team_home',
       'score_home', 'score_away', 'team_away', 'team_favorite_id',
       'spread_favorite', 'over_under_line', 'stadium', 'stadium_neutral',
       'weather_temperature', 'weather_wind_mph', 'weather_humidity',
       'weather_detail', 'winner', 'over', 'home_abrev', 'away_abrev'],
      dtype='object')

In [11]:
#since we want to build a predictive model drop al of the posterior values
master_df.drop(['elo1_post',"elo2_post","qb1_value_post","qb2_value_post","qb1_game_value","qb2_game_value","qbelo1_post","qbelo2_post" ], axis=1,inplace=True)

In [12]:

def find_point_diff(df):
    '''
    function to find the actual point differential in a game. 
    define the point differential as favored team points - other team points
    This function is supposed to be applied to a dataframe
    '''
    if df['team_favorite_id'] == df['home_abrev']:
        return df['score_home'] - df['score_away']
    elif df['team_favorite_id'] == df['away_abrev']:
        return df['score_away'] - df['score_home']
    else:
        return np.nan
 
games['point_diff'] = games.apply(find_point_diff,axis=1)
games['point_total'] = games['score_home'] + games['score_away']
games

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,winner,over,home_abrev,away_abrev,gameID,point_diff,point_total
350,1968-01-14 00:00:00,1967,Superbowl,True,Green Bay Packers,33.0,14.0,Oakland Raiders,GB,-13.5,43.0,Orange Bowl,True,60.0,12.0,74.0,NaN,Green Bay Packers,1,GB,LVR,1968-01-14 00:00:00 GB vs LVR,19.0,47.0
538,1969-01-12 00:00:00,1968,Superbowl,True,Baltimore Colts,7.0,16.0,New York Jets,IND,-18.0,40.0,Orange Bowl,True,66.0,12.0,80.0,rain,New York Jets,0,IND,NYJ,1969-01-12 00:00:00 IND vs NYJ,-9.0,23.0
727,1970-01-11 00:00:00,1969,Superbowl,True,Kansas City Chiefs,23.0,7.0,Minnesota Vikings,MIN,-12.0,39.0,Tulane Stadium,True,55.0,14.0,84.0,rain,Kansas City Chiefs,0,KC,MIN,1970-01-11 00:00:00 KC vs MIN,-16.0,30.0
916,1971-01-17 00:00:00,1970,Superbowl,True,Baltimore Colts,16.0,13.0,Dallas Cowboys,IND,-2.5,36.0,Orange Bowl,True,59.0,11.0,60.0,NaN,Baltimore Colts,0,IND,DAL,1971-01-17 00:00:00 DAL vs IND,3.0,29.0
1105,1972-01-16 00:00:00,1971,Superbowl,True,Dallas Cowboys,24.0,3.0,Miami Dolphins,DAL,-6.0,34.0,Tulane Stadium,True,34.0,18.0,40.0,NaN,Dallas Cowboys,0,DAL,MIA,1972-01-16 00:00:00 DAL vs MIA,21.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13544,2023-09-17 00:00:00,2023,2,False,Tampa Bay Buccaneers,27.0,17.0,Chicago Bears,TB,-2.5,41.0,Raymond James Stadium,False,NaN,NaN,NaN,NaN,Tampa Bay Buccaneers,1,TB,CHI,2023-09-17 00:00:00 CHI vs TB,10.0,44.0
13545,2023-09-17 00:00:00,2023,2,False,Tennessee Titans,27.0,24.0,Los Angeles Chargers,LAC,-2.5,45.5,Nissan Stadium,False,NaN,NaN,NaN,NaN,Tennessee Titans,1,TEN,LAC,2023-09-17 00:00:00 LAC vs TEN,-3.0,51.0
13546,2023-09-18 00:00:00,2023,2,False,Carolina Panthers,17.0,20.0,New Orleans Saints,NO,-3.0,39.5,Bank of America Stadium,False,NaN,NaN,NaN,NaN,New Orleans Saints,0,CAR,NO,2023-09-18 00:00:00 CAR vs NO,3.0,37.0
13547,2023-09-18 00:00:00,2023,2,False,Pittsburgh Steelers,26.0,22.0,Cleveland Browns,CLE,-2.0,38.0,Acrisure Stadium,False,NaN,NaN,NaN,NaN,Pittsburgh Steelers,1,PIT,CLE,2023-09-18 00:00:00 CLE vs PIT,-4.0,48.0


### Scraping

The boxscores from previous games could be useful information to have. Let's write a scraper to get every boxscore from 1978 to today from footballdb.com

In [82]:
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15'
    ]

In [6]:
# You don't need to run this code anymore just read from links.txt
years = list(range(1978,2024))
'''
use a different user agent and proxy for each request so we don't get banned since we are sending so many requests
first scrape all of the links to the boxscores
'''
with open('Data/http_proxies.txt', 'r') as file:
    ip_list = file.readlines()
links = []
for year in years:
    header = {"User-Agent": user_agents[randint(0,len(user_agents) - 1)]}
    # proxy = ip_list[randint(0,len(ip_list)-1)]
    # proxies = {
    #         "http":f"'http://{proxy[:-1]}",
    #         "https":f"https://{proxy[:-1]}"
    #         }
    r = requests.get("https://www.footballdb.com/games/index.html",params={"lg":"NFL","yr":year},headers=header)
    soup = BeautifulSoup(r.content,"html.parser")
    tables = soup.find_all("table",class_ = "statistics")
    tables

    for table in tables:
        tbl_links = (table.find_all("a", href = True))
        for link in tbl_links:
            links.append("https://www.footballdb.com/"+link['href'])


In [7]:
#write links to a file so we don't need to scrape multiple times and risk getting IP banned
with open("links.txt",'w') as fp:
    for link in links:
        fp.write(f"{link}\n")
    fp.close()


In [215]:
columns = {'Att - Comp - Int':['passing-attempts','completions','int-thrown'],
           'Interception Returns':['interceptions', 'int-return-yards'],
           'Fumbles - Lost':['fumbles','fumbles-lost'],
           'Field Goals': ['fga','fgm'],
           'Third Downs': ['3rd-down-convs','3rd-downs','3rd-down-conv-rate'],
           'Punts - Average': ['punts','yards-per-punt'],
           'Penalties - Yards':['penalties','penalty-yards'],
           'Sacked - Yds Lost':['sacks_allowed','sack_yds_lost'],
           'Punt Returns':['punts-returned','punt-return-yds'],
           'Kickoff Returns':['kicks-returned','kick-return-yds'],
            }
with open("links.txt",'r') as fp:
    urls = fp.readlines()
    fp.close()

In [324]:
def parse_req(r,date,matchup):
    #parse the request to get the the box score tables
    soup = BeautifulSoup(r.content,'html.parser')
    stats = soup.find('div',id='divBox_team')
    table = stats.find_all('table',class_ = 'statistics')
    table_str = io.StringIO(str(table))
    tables = pd.read_html(table_str)
    pre_box_score = pd.concat(tables)
    #pre_box_score['gamedID'] = gameid
    pre_box_score.set_index('Unnamed: 0',inplace=True)
    box_score = pre_box_score.T
    box_score.reset_index(inplace=True)
    box_score.rename_axis(None, axis=1,inplace=True)
    box_score['date'] = date
    box_score['matchup'] = matchup
    if "Time of Possesion" not in box_score.columns:
        box_score['Time of Possession'] = np.nan
    if "Fourth Downs" not in box_score.columns:
        box_score["Fourth Downs"] = np.nan
    if len(box_score.columns) != len(list(set(box_score.columns))):
        seen = set()
        dupes = []

        for x in box_score.columns:
            if x in seen:
                dupes.append(x)
            else:
                seen.add(x)
        new_cols = []
        count = 1
        for col in box_score.columns:
            if col in dupes:
                new_cols.append(f"{col}_{count}")
                count += 1
            else:
                new_cols.append(col)
        box_score.columns = new_cols    
    
    pd.to_datetime(box_score.date)
    return box_score

In [312]:
def scrape_box_score(url):
    #links to all the box scores on footballDB
    #check to see if the loop has run already
    #parse out the date and matchup info from the URL
    matchup_date = url.split('/')[-1][:-3]
    date = matchup_date.split('-')[-1]
    date = date[:4] + '-' + date[4:6]+ '-'+ date[6:8]
    matchup = " ".join([str(item) for item in matchup_date.split('-')[:-1]])
    #request and parse data into a DF using parse_req()
    header = {"User-Agent": user_agents[randint(0,len(user_agents) - 1)]}
    r = requests.get(url,headers=header)
    box_score = parse_req(r,date,matchup)
    return box_score

In [340]:
box_scores = scrape_box_score(urls[0])
for url in urls[1:]:
    box_score = scrape_box_score(url)
    box_scores = pd.concat([box_scores,box_score],join='outer',axis=0)
    box_scores.reset_index(inplace=True,drop=True)
box_scores

,index,First downs,Rushing,Passing,Penalty,Total Net Yards,Net Yards Rushing,Rushing Plays,Average Gain_1,Net Yards Passing,Att - Comp - Int,Sacked - Yds Lost,Gross Yards Passing,Avg. Yds/Att,Punts - Average,Had Blocked,Punt Returns,Kickoff Returns,Interception Returns,Penalties - Yards,Fumbles - Lost,Field Goals,Third Downs,Total Plays,Average Gain_2,date,matchup,Time of Possession,Fourth Downs
0,NY GiantsNYG,12,6,6,0,238,76,35,2.2,162,25-12-1,4-21,183,7.3,9-43.3,0,4--11,4-103,3-46,7-64,0-0,2-2,5-18-27%,64,3.7,1978-09-02,new york giants vs tampa bay buccaneers,NaN,NaN
1,Tampa BayTB,16,9,4,3,251,165,39,4.2,86,28-10-3,1-7,93,3.3,7-44.3,0,5-76,5-86,1-3,8-55,4-1,2-2,4-17-23%,68,3.7,1978-09-02,new york giants vs tampa bay buccaneers,NaN,NaN
2,Green BayGB,16,11,2,3,212,181,55,3.3,31,9-3-0,0-0,31,3.4,7-40.6,0,2-15,2-45,1-10,8-40,1-0,2-2,8-18-44%,64,3.3,1978-09-03,green bay packers vs detroit lions,NaN,NaN
3,DetroitDet,11,5,6,0,122,82,26,3.2,40,19-10-1,8-76,116,6.1,7-38.7,0,5-46,4-91,-,9-31,3-0,0-0,4-15-26%,53,2.3,1978-09-03,green bay packers vs detroit lions,NaN,NaN
4,HoustonHou,13,2,9,2,261,156,21,7.4,105,36-19-2,4-31,136,3.8,9-37.1,1,4-23,5-169,-,7-52,4-2,0-0,4-15-26%,61,4.3,1978-09-03,houston oilers vs atlanta falcons,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22935,PhiladelphiaPhi,24,9,12,3,378,185,32,5.8,193,31-18-1,2-7,200,6.5,5-53.8,0,1-18,0-0,1-5,4-30,1-1,1-1,4-11-36%,65,5.8,2023-11-26,buffalo bills vs philadelphia eagles,NaN,0-0-0%
22936,BaltimoreBal,20,10,9,1,361,197,35,5.6,164,32-18-0,2-13,177,5.5,3-40.3,0,1-3,0-0,1-0,5-42,1-0,2-3,4-13-30%,69,5.2,2023-11-26,baltimore ravens vs los angeles chargers,NaN,1-3-33%
22937,LA ChargersLAC,16,5,11,0,279,86,19,4.5,193,44-29-1,3-24,217,4.9,3-52.0,0,1-23,1-21,0-0,5-41,3-3,1-1,7-15-46%,66,4.2,2023-11-26,baltimore ravens vs los angeles chargers,NaN,1-3-33%
22938,ChicagoChi,18,6,12,0,317,118,28,4.2,199,37-27-0,3-18,217,5.9,3-45.7,0,1-0,1-28,4-19,7-76,3-2,4-5,8-18-44%,68,4.7,2023-11-27,chicago bears vs minnesota vikings,NaN,1-1-100%


In [341]:
box_scores_csv = box_scores.to_csv()

In [344]:
with open("box_scores.csv", "w") as fp:
    fp.write(box_scores_csv)
    fp.close()

In [13]:
box_scores_df = pd.read_csv('box_scores.csv')
box_scores_df

,Unnamed: 0,index,First downs,Rushing,Passing,Penalty,Total Net Yards,Net Yards Rushing,Rushing Plays,Average Gain_1,Net Yards Passing,Att - Comp - Int,Sacked - Yds Lost,Gross Yards Passing,Avg. Yds/Att,Punts - Average,Had Blocked,Punt Returns,Kickoff Returns,Interception Returns,Penalties - Yards,Fumbles - Lost,Field Goals,Third Downs,Total Plays,Average Gain_2,date,matchup,Time of Possession,Fourth Downs
0,0,NY GiantsNYG,12,6,6,0,238,76,35,2.2,162,25-12-1,4-21,183,7.3,9-43.3,0.0,4--11,4-103,3-46,7-64,0-0,2-2,5-18-27%,64,3.7,1978-09-02,new york giants vs tampa bay buccaneers,NaN,NaN
1,1,Tampa BayTB,16,9,4,3,251,165,39,4.2,86,28-10-3,1-7,93,3.3,7-44.3,0.0,5-76,5-86,1-3,8-55,4-1,2-2,4-17-23%,68,3.7,1978-09-02,new york giants vs tampa bay buccaneers,NaN,NaN
2,2,Green BayGB,16,11,2,3,212,181,55,3.3,31,9-3-0,0-0,31,3.4,7-40.6,0.0,2-15,2-45,1-10,8-40,1-0,2-2,8-18-44%,64,3.3,1978-09-03,green bay packers vs detroit lions,NaN,NaN
3,3,DetroitDet,11,5,6,0,122,82,26,3.2,40,19-10-1,8-76,116,6.1,7-38.7,0.0,5-46,4-91,-,9-31,3-0,0-0,4-15-26%,53,2.3,1978-09-03,green bay packers vs detroit lions,NaN,NaN
4,4,HoustonHou,13,2,9,2,261,156,21,7.4,105,36-19-2,4-31,136,3.8,9-37.1,1.0,4-23,5-169,-,7-52,4-2,0-0,4-15-26%,61,4.3,1978-09-03,houston oilers vs atlanta falcons,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22935,22935,PhiladelphiaPhi,24,9,12,3,378,185,32,5.8,193,31-18-1,2-7,200,6.5,5-53.8,0.0,1-18,0-0,1-5,4-30,1-1,1-1,4-11-36%,65,5.8,2023-11-26,buffalo bills vs philadelphia eagles,NaN,0-0-0%
22936,22936,BaltimoreBal,20,10,9,1,361,197,35,5.6,164,32-18-0,2-13,177,5.5,3-40.3,0.0,1-3,0-0,1-0,5-42,1-0,2-3,4-13-30%,69,5.2,2023-11-26,baltimore ravens vs los angeles chargers,NaN,1-3-33%
22937,22937,LA ChargersLAC,16,5,11,0,279,86,19,4.5,193,44-29-1,3-24,217,4.9,3-52.0,0.0,1-23,1-21,0-0,5-41,3-3,1-1,7-15-46%,66,4.2,2023-11-26,baltimore ravens vs los angeles chargers,NaN,1-3-33%
22938,22938,ChicagoChi,18,6,12,0,317,118,28,4.2,199,37-27-0,3-18,217,5.9,3-45.7,0.0,1-0,1-28,4-19,7-76,3-2,4-5,8-18-44%,68,4.7,2023-11-27,chicago bears vs minnesota vikings,NaN,1-1-100%


This Data is not tidy, let's clean it up

In [46]:
#clean the data up
columns = {'Att - Comp - Int':['passing-attempts','completions','int-thrown'],
           'Interception Returns':['interceptions', 'int-return-yards'],
           'Fumbles - Lost':['fumbles','fumbles-lost'],
           'Field Goals': ['fga','fgm'],
           'Third Downs': ['3rd-down-convs','3rd-downs','3rd-down-conv-rate'],
           'Punts - Average': ['punts','yards-per-punt'],
           'Penalties - Yards':['penalties','penalty-yards'],
           'Sacked - Yds Lost':['sacks_allowed','sack-yds-lost'],
           'Punt Returns':['punts-returned','punt-return-yds'],
           'Kickoff Returns':['kicks-returned','kick-return-yds'],

            }
#'Fourth Downs':['4th-down-convs','4th-downs','4th-down-conv-rate'],
#First rename some columns 
box_scores_df.rename(columns={"index":"team",
                          "First downs":"total-first-downs",
                          "Rushing": "rushing-first-downs",
                          "Passing": "passing-first-downs",
                          "Penalty": "penalty-first-downs",
                          "Average Gain_1": "avg-gain-rushing",
                          "Avg. Yards/Att": "yards-per-att",
                          "Rushing Plays": "rushing-plays",
                          "Total Net Yards": "net-yards",
                          "Net Yards Rushing": "net-rushing-yds",
                          "Net Yards Passing": "net-passing-yds",
                          "Gross Yards Passing":"gross-passing-yds",
                          "Avg. Yds/Att": "yds-per-att",
                          "Had Blocked": "blocked-kicks-allowed",
                          "Time of Possesion": "time-of-possession",
                          "Total Plays": "total-plays",
                          "Average Gain_2": "avg-gain-per-play",
                          },inplace=True)
dropped_cols = ['Punts - Average','Penalties - Yards','Third Downs','Field Goals','Fumbles - Lost','Interception Returns','Att - Comp - Int','Penalties - Yards','Sacked - Yds Lost','Punt Returns','Kickoff Returns','Fourth Downs']

In [91]:
#fix typos
box_scores_df.replace({"--":"-", "":"0"},regex=True, inplace=True)

In [84]:
#split the columns with multiple values into different columns
for col in columns.keys():
    box_scores_df[columns[col]] = box_scores_df[col].str.split('-',expand=True)
box_scores_df.drop(columns.keys(),axis =1, inplace=True)
box_scores_df.drop(["Unnamed: 0"],axis =1, inplace = True)

In [ ]:
#change column types to float
cols_to_float = list(box_scores_df.columns)
cols_to_float.remove("team")
cols_to_float.remove("matchup")
cols_to_float.remove("date")
cols_to_float.remove("Fourth Downs")
box_scores_df['3rd-down-conv-rate']= box_scores_df['3rd-down-conv-rate'].str.strip('%')
box_scores_df[cols_to_float] = box_scores_df[cols_to_float].astype(float)
box_scores_df

In [138]:
box_scores_df['3rd-down-conv-rate'] = box_scores_df['3rd-down-conv-rate'] / 100

In [136]:
box_scores_cleaned_csv = box_scores_df.to_csv()
with open("Data/box_scores_cleaned.csv", "w") as fp:
    fp.write(box_scores_cleaned_csv)
    fp.close()

Now that we have our cleaned box scores, we need to find a way to make this data useful. Right now all we have is data that can be collected after a game is over, but if we want to build a predictive model that is of no use to us. We need data that can be collected before the game starts. However, we can use data from previous games to get an idea of how a team performs on average. For example if we want to look at how a team performs on average on third downs we can look at the average from their previous 5, 10, 15, etc. games.

In [187]:
box_scores_df = pd.read_csv("Data/box_scores_cleaned.csv")
box_scores_df['team-abrev'].replace({"AK":"LVR"},inplace=True)


In [188]:
rolling_net_yards_16_games = box_scores_df.groupby('team')['net-yards'].rolling(window = 16,min_periods = 1).mean().reset_index(level=0)
box_scores_df = box_scores_df.join(rolling_net_yards_16_games['net-yards'],rsuffix="-rolling-16")
box_scores_df

,Unnamed: 0,team,total-first-downs,rushing-first-downs,passing-first-downs,penalty-first-downs,net-yards,net-rushing-yds,rushing-plays,avg-gain-rushing,net-passing-yds,gross-passing-yds,yds-per-att,blocked-kicks-allowed,total-plays,avg-gain-per-play,date,matchup,Time of Possession,Fourth Downs,passing-attempts,completions,int-thrown,interceptions,int-return-yards,fumbles,fumbles-lost,fga,fgm,3rd-down-convs,3rd-downs,3rd-down-conv-rate,punts,yards-per-punt,penalties,penalty-yards,sacks_allowed,sack-yds-lost,punts-returned,punt-return-yds,kicks-returned,kick-return-yds,Third Downs,team-abrev,net-yards-rolling-16
0,0,NY GiantsNYG,12.0,6.0,6.0,0.0,238.0,76.0,35.0,2.2,162.0,183.0,7.3,0.0,64.0,3.7,1978-09-02,new york giants vs tampa bay buccaneers,NaN,NaN,25.0,12.0,1.0,3.0,46.0,0.0,0.0,2.0,2.0,5.0,18.0,27.0,9.0,43.3,7.0,64.0,4.0,21.0,4.0,11.0,4.0,103.0,27.0,NYG,238.0000
1,1,Tampa BayTB,16.0,9.0,4.0,3.0,251.0,165.0,39.0,4.2,86.0,93.0,3.3,0.0,68.0,3.7,1978-09-02,new york giants vs tampa bay buccaneers,NaN,NaN,28.0,10.0,3.0,1.0,3.0,4.0,1.0,2.0,2.0,4.0,17.0,23.0,7.0,44.3,8.0,55.0,1.0,7.0,5.0,76.0,5.0,86.0,23.0,TB,251.0000
2,2,Green BayGB,16.0,11.0,2.0,3.0,212.0,181.0,55.0,3.3,31.0,31.0,3.4,0.0,64.0,3.3,1978-09-03,green bay packers vs detroit lions,NaN,NaN,9.0,3.0,0.0,1.0,10.0,1.0,0.0,2.0,2.0,8.0,18.0,44.0,7.0,40.6,8.0,40.0,0.0,0.0,2.0,15.0,2.0,45.0,44.0,GB,212.0000
3,3,DetroitDet,11.0,5.0,6.0,0.0,122.0,82.0,26.0,3.2,40.0,116.0,6.1,0.0,53.0,2.3,1978-09-03,green bay packers vs detroit lions,NaN,NaN,19.0,10.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,15.0,26.0,7.0,38.7,9.0,31.0,8.0,76.0,5.0,46.0,4.0,91.0,26.0,DET,122.0000
4,4,HoustonHou,13.0,2.0,9.0,2.0,261.0,156.0,21.0,7.4,105.0,136.0,3.8,1.0,61.0,4.3,1978-09-03,houston oilers vs atlanta falcons,NaN,NaN,36.0,19.0,2.0,0.0,0.0,4.0,2.0,0.0,0.0,4.0,15.0,26.0,9.0,37.1,7.0,52.0,4.0,31.0,4.0,23.0,5.0,169.0,26.0,HOU,261.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22935,22935,PhiladelphiaPhi,24.0,9.0,12.0,3.0,378.0,185.0,32.0,5.8,193.0,200.0,6.5,0.0,65.0,5.8,2023-11-26,buffalo bills vs philadelphia eagles,NaN,0-0-0%,31.0,18.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,4.0,11.0,36.0,5.0,53.8,4.0,30.0,2.0,7.0,1.0,18.0,0.0,0.0,36.0,PHI,360.2500
22936,22936,BaltimoreBal,20.0,10.0,9.0,1.0,361.0,197.0,35.0,5.6,164.0,177.0,5.5,0.0,69.0,5.2,2023-11-26,baltimore ravens vs los angeles chargers,NaN,1-3-33%,32.0,18.0,0.0,1.0,0.0,1.0,0.0,2.0,3.0,4.0,13.0,30.0,3.0,40.3,5.0,42.0,2.0,13.0,1.0,3.0,0.0,0.0,30.0,BAL,355.1250
22937,22937,LA ChargersLAC,16.0,5.0,11.0,0.0,279.0,86.0,19.0,4.5,193.0,217.0,4.9,0.0,66.0,4.2,2023-11-26,baltimore ravens vs los angeles chargers,NaN,1-3-33%,44.0,29.0,1.0,0.0,0.0,3.0,3.0,1.0,1.0,7.0,15.0,46.0,3.0,52.0,5.0,41.0,3.0,24.0,1.0,23.0,1.0,21.0,46.0,LAC,350.2500
22938,22938,ChicagoChi,18.0,6.0,12.0,0.0,317.0,118.0,28.0,4.2,199.0,217.0,5.9,0.0,68.0,4.7,2023-11-27,chicago bears vs minnesota vikings,NaN,1-1-100%,37.0,27.0,0.0,4.0,19.0,3.0,2.0,4.0,5.0,8.0,18.0,44.0,3.0,45.7,7.0,76.0,3.0,18.0,1.0,0.0,1.0,28.0,44.0,CHI,301.5625


In [196]:
def get_rolling(column,window_size):
    return box_scores_df.groupby('team')[column].rolling(window = window_size,min_periods = 1).mean().reset_index(level=0)[column]

In [198]:
box_scores_df['total-first-downs-rolling-16'] = get_rolling(column='total-first-downs',window_size=16)

In [199]:
box_scores_df

,Unnamed: 0,team,total-first-downs,rushing-first-downs,passing-first-downs,penalty-first-downs,net-yards,net-rushing-yds,rushing-plays,avg-gain-rushing,net-passing-yds,gross-passing-yds,yds-per-att,blocked-kicks-allowed,total-plays,avg-gain-per-play,date,matchup,Time of Possession,Fourth Downs,passing-attempts,completions,int-thrown,interceptions,int-return-yards,fumbles,fumbles-lost,fga,fgm,3rd-down-convs,3rd-downs,3rd-down-conv-rate,punts,yards-per-punt,penalties,penalty-yards,sacks_allowed,sack-yds-lost,punts-returned,punt-return-yds,kicks-returned,kick-return-yds,Third Downs,team-abrev,net-yards-rolling-16,total-first-downs-rolling-16
0,0,NY GiantsNYG,12.0,6.0,6.0,0.0,238.0,76.0,35.0,2.2,162.0,183.0,7.3,0.0,64.0,3.7,1978-09-02,new york giants vs tampa bay buccaneers,NaN,NaN,25.0,12.0,1.0,3.0,46.0,0.0,0.0,2.0,2.0,5.0,18.0,27.0,9.0,43.3,7.0,64.0,4.0,21.0,4.0,11.0,4.0,103.0,27.0,NYG,238.0000,12.0000
1,1,Tampa BayTB,16.0,9.0,4.0,3.0,251.0,165.0,39.0,4.2,86.0,93.0,3.3,0.0,68.0,3.7,1978-09-02,new york giants vs tampa bay buccaneers,NaN,NaN,28.0,10.0,3.0,1.0,3.0,4.0,1.0,2.0,2.0,4.0,17.0,23.0,7.0,44.3,8.0,55.0,1.0,7.0,5.0,76.0,5.0,86.0,23.0,TB,251.0000,16.0000
2,2,Green BayGB,16.0,11.0,2.0,3.0,212.0,181.0,55.0,3.3,31.0,31.0,3.4,0.0,64.0,3.3,1978-09-03,green bay packers vs detroit lions,NaN,NaN,9.0,3.0,0.0,1.0,10.0,1.0,0.0,2.0,2.0,8.0,18.0,44.0,7.0,40.6,8.0,40.0,0.0,0.0,2.0,15.0,2.0,45.0,44.0,GB,212.0000,16.0000
3,3,DetroitDet,11.0,5.0,6.0,0.0,122.0,82.0,26.0,3.2,40.0,116.0,6.1,0.0,53.0,2.3,1978-09-03,green bay packers vs detroit lions,NaN,NaN,19.0,10.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,15.0,26.0,7.0,38.7,9.0,31.0,8.0,76.0,5.0,46.0,4.0,91.0,26.0,DET,122.0000,11.0000
4,4,HoustonHou,13.0,2.0,9.0,2.0,261.0,156.0,21.0,7.4,105.0,136.0,3.8,1.0,61.0,4.3,1978-09-03,houston oilers vs atlanta falcons,NaN,NaN,36.0,19.0,2.0,0.0,0.0,4.0,2.0,0.0,0.0,4.0,15.0,26.0,9.0,37.1,7.0,52.0,4.0,31.0,4.0,23.0,5.0,169.0,26.0,HOU,261.0000,13.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22935,22935,PhiladelphiaPhi,24.0,9.0,12.0,3.0,378.0,185.0,32.0,5.8,193.0,200.0,6.5,0.0,65.0,5.8,2023-11-26,buffalo bills vs philadelphia eagles,NaN,0-0-0%,31.0,18.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,4.0,11.0,36.0,5.0,53.8,4.0,30.0,2.0,7.0,1.0,18.0,0.0,0.0,36.0,PHI,360.2500,22.5000
22936,22936,BaltimoreBal,20.0,10.0,9.0,1.0,361.0,197.0,35.0,5.6,164.0,177.0,5.5,0.0,69.0,5.2,2023-11-26,baltimore ravens vs los angeles chargers,NaN,1-3-33%,32.0,18.0,0.0,1.0,0.0,1.0,0.0,2.0,3.0,4.0,13.0,30.0,3.0,40.3,5.0,42.0,2.0,13.0,1.0,3.0,0.0,0.0,30.0,BAL,355.1250,20.2500
22937,22937,LA ChargersLAC,16.0,5.0,11.0,0.0,279.0,86.0,19.0,4.5,193.0,217.0,4.9,0.0,66.0,4.2,2023-11-26,baltimore ravens vs los angeles chargers,NaN,1-3-33%,44.0,29.0,1.0,0.0,0.0,3.0,3.0,1.0,1.0,7.0,15.0,46.0,3.0,52.0,5.0,41.0,3.0,24.0,1.0,23.0,1.0,21.0,46.0,LAC,350.2500,19.6875
22938,22938,ChicagoChi,18.0,6.0,12.0,0.0,317.0,118.0,28.0,4.2,199.0,217.0,5.9,0.0,68.0,4.7,2023-11-27,chicago bears vs minnesota vikings,NaN,1-1-100%,37.0,27.0,0.0,4.0,19.0,3.0,2.0,4.0,5.0,8.0,18.0,44.0,3.0,45.7,7.0,76.0,3.0,18.0,1.0,0.0,1.0,28.0,44.0,CHI,301.5625,17.5625


In [108]:
#Create a new column team-abrev with the abreviation of the team
box_scores_df['team-abrev'] = box_scores_df['team'].str[-3:]
def check_team_abrev(df):
    id = df['team-abrev']
    id = id.upper()
    if id in list(teams['team_id']):
        return id
    else:
        return id[1:]
box_scores_df['team-abrev'] = box_scores_df.apply(check_team_abrev,axis=1)

In [128]:
#Merge every other row so we can have the box scores for both teams in one row
team1_box_scores = box_scores_df[box_scores_df.index % 2 == 0]
team2_box_scores = box_scores_df[box_scores_df.index % 2 == 1]
team1_box_scores.reset_index(inplace=True,drop=True)
team2_box_scores.reset_index(inplace=True,drop=True)
team2_box_scores
box_scores_by_game = team1_box_scores.join(team2_box_scores,how="outer",lsuffix="-1",rsuffix="-2")

In [129]:
box_scores_by_game

,team-1,total-first-downs-1,rushing-first-downs-1,passing-first-downs-1,penalty-first-downs-1,net-yards-1,net-rushing-yds-1,rushing-plays-1,avg-gain-rushing-1,net-passing-yds-1,gross-passing-yds-1,yds-per-att-1,blocked-kicks-allowed-1,total-plays-1,avg-gain-per-play-1,date-1,matchup-1,Time of Possession-1,Fourth Downs-1,passing-attempts-1,completions-1,int-thrown-1,interceptions-1,int-return-yards-1,fumbles-1,fumbles-lost-1,fga-1,fgm-1,3rd-down-convs-1,3rd-downs-1,3rd-down-conv-rate-1,punts-1,yards-per-punt-1,penalties-1,penalty-yards-1,sacks_allowed-1,sack-yds-lost-1,punts-returned-1,punt-return-yds-1,kicks-returned-1,kick-return-yds-1,Third Downs-1,team-abrev-1,team-2,total-first-downs-2,rushing-first-downs-2,passing-first-downs-2,penalty-first-downs-2,net-yards-2,net-rushing-yds-2,rushing-plays-2,avg-gain-rushing-2,net-passing-yds-2,gross-passing-yds-2,yds-per-att-2,blocked-kicks-allowed-2,total-plays-2,avg-gain-per-play-2,date-2,matchup-2,Time of Possession-2,Fourth Downs-2,passing-attempts-2,completions-2,int-thrown-2,interceptions-2,int-return-yards-2,fumbles-2,fumbles-lost-2,fga-2,fgm-2,3rd-down-convs-2,3rd-downs-2,3rd-down-conv-rate-2,punts-2,yards-per-punt-2,penalties-2,penalty-yards-2,sacks_allowed-2,sack-yds-lost-2,punts-returned-2,punt-return-yds-2,kicks-returned-2,kick-return-yds-2,Third Downs-2,team-abrev-2
0,NY GiantsNYG,12.0,6.0,6.0,0.0,238.0,76.0,35.0,2.2,162.0,183.0,7.3,0.0,64.0,3.7,1978-09-02,new york giants vs tampa bay buccaneers,NaN,NaN,25.0,12.0,1.0,3.0,46.0,0.0,0.0,2.0,2.0,5.0,18.0,27.0,9.0,43.3,7.0,64.0,4.0,21.0,4.0,11.0,4.0,103.0,27.0,NYG,Tampa BayTB,16.0,9.0,4.0,3.0,251.0,165.0,39.0,4.2,86.0,93.0,3.3,0.0,68.0,3.7,1978-09-02,new york giants vs tampa bay buccaneers,NaN,NaN,28.0,10.0,3.0,1.0,3.0,4.0,1.0,2.0,2.0,4.0,17.0,23.0,7.0,44.3,8.0,55.0,1.0,7.0,5.0,76.0,5.0,86.0,23.0,TB
1,Green BayGB,16.0,11.0,2.0,3.0,212.0,181.0,55.0,3.3,31.0,31.0,3.4,0.0,64.0,3.3,1978-09-03,green bay packers vs detroit lions,NaN,NaN,9.0,3.0,0.0,1.0,10.0,1.0,0.0,2.0,2.0,8.0,18.0,44.0,7.0,40.6,8.0,40.0,0.0,0.0,2.0,15.0,2.0,45.0,44.0,GB,DetroitDet,11.0,5.0,6.0,0.0,122.0,82.0,26.0,3.2,40.0,116.0,6.1,0.0,53.0,2.3,1978-09-03,green bay packers vs detroit lions,NaN,NaN,19.0,10.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,15.0,26.0,7.0,38.7,9.0,31.0,8.0,76.0,5.0,46.0,4.0,91.0,26.0,DET
2,HoustonHou,13.0,2.0,9.0,2.0,261.0,156.0,21.0,7.4,105.0,136.0,3.8,1.0,61.0,4.3,1978-09-03,houston oilers vs atlanta falcons,NaN,NaN,36.0,19.0,2.0,0.0,0.0,4.0,2.0,0.0,0.0,4.0,15.0,26.0,9.0,37.1,7.0,52.0,4.0,31.0,4.0,23.0,5.0,169.0,26.0,HOU,AtlantaAtl,11.0,6.0,4.0,1.0,171.0,100.0,35.0,2.9,71.0,104.0,4.5,0.0,62.0,2.8,1978-09-03,houston oilers vs atlanta falcons,NaN,NaN,23.0,10.0,0.0,2.0,14.0,3.0,1.0,2.0,3.0,2.0,14.0,14.0,9.0,42.7,8.0,75.0,4.0,33.0,6.0,59.0,3.0,80.0,14.0,ATL
3,Kansas CityKC,26.0,14.0,9.0,3.0,388.0,267.0,69.0,3.9,121.0,133.0,6.7,1.0,90.0,4.3,1978-09-03,kansas city chiefs vs cincinnati bengals,NaN,NaN,20.0,14.0,0.0,2.0,0.0,4.0,2.0,1.0,1.0,12.0,21.0,57.0,5.0,36.2,9.0,82.0,1.0,12.0,2.0,16.0,4.0,104.0,57.0,KC,CincinnatiCin,12.0,3.0,8.0,1.0,205.0,77.0,17.0,4.5,128.0,159.0,6.6,0.0,45.0,4.6,1978-09-03,kansas city chiefs vs cincinnati bengals,NaN,NaN,24.0,10.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,4.0,10.0,40.0,4.0,35.8,4.0,30.0,4.0,31.0,4.0,29.0,5.0,89.0,40.0,CIN
4,Los AngelesLA,15.0,5.0,9.0,1.0,220.0,103.0,33.0,3.1,117.0,154.0,4.7,0.0,71.0,3.1,1978-09-03,los angeles rams vs philadelphia eagles,NaN,NaN,33.0,14.0,0.0,1.0,8.0,3.0,1.0,3.0,4.0,5.0,15.0,33.0,9.0,43.3,10.0,82.0,5.0,37.0,4.0,26.0,3.0,45.0,33.0,LA,PhiladelphiaPhi,10.0,5.0,4.0,1.0,208.0,128.0,30.0,4.3,80.0,102.0,6.0,1.0,50.0,4.2,1978-09-03,los angeles rams vs philadelphia eagles,NaN,NaN,17.0,7.0,1.0,0.0,0.0,2.0,1.0,0.0,1.0,3.0,13.0,23.0,8.0,36.8,3.0,38.0,3.0,22.0,7.0,76.0,2.0,48.0,23.0,PHI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [130]:
def make_game_id_3(box_scores_DF):
    team1 = box_scores_DF['team-abrev-1']
    team2 = box_scores_DF['team-abrev-2']
    teams = [team1, team2]
    sorted_teams = sorted(teams)
    date_str = str(box_scores_DF['date-1'])
    gameID = date_str + ' ' + sorted_teams[0] + ' vs ' + sorted_teams[1]
    return gameID
box_scores_by_game['game-id'] = box_scores_by_game.apply(make_game_id_3,axis=1)
box_scores_by_game

,team-1,total-first-downs-1,rushing-first-downs-1,passing-first-downs-1,penalty-first-downs-1,net-yards-1,net-rushing-yds-1,rushing-plays-1,avg-gain-rushing-1,net-passing-yds-1,gross-passing-yds-1,yds-per-att-1,blocked-kicks-allowed-1,total-plays-1,avg-gain-per-play-1,date-1,matchup-1,Time of Possession-1,Fourth Downs-1,passing-attempts-1,completions-1,int-thrown-1,interceptions-1,int-return-yards-1,fumbles-1,fumbles-lost-1,fga-1,fgm-1,3rd-down-convs-1,3rd-downs-1,3rd-down-conv-rate-1,punts-1,yards-per-punt-1,penalties-1,penalty-yards-1,sacks_allowed-1,sack-yds-lost-1,punts-returned-1,punt-return-yds-1,kicks-returned-1,kick-return-yds-1,Third Downs-1,team-abrev-1,team-2,total-first-downs-2,rushing-first-downs-2,passing-first-downs-2,penalty-first-downs-2,net-yards-2,net-rushing-yds-2,rushing-plays-2,avg-gain-rushing-2,net-passing-yds-2,gross-passing-yds-2,yds-per-att-2,blocked-kicks-allowed-2,total-plays-2,avg-gain-per-play-2,date-2,matchup-2,Time of Possession-2,Fourth Downs-2,passing-attempts-2,completions-2,int-thrown-2,interceptions-2,int-return-yards-2,fumbles-2,fumbles-lost-2,fga-2,fgm-2,3rd-down-convs-2,3rd-downs-2,3rd-down-conv-rate-2,punts-2,yards-per-punt-2,penalties-2,penalty-yards-2,sacks_allowed-2,sack-yds-lost-2,punts-returned-2,punt-return-yds-2,kicks-returned-2,kick-return-yds-2,Third Downs-2,team-abrev-2,game-id
0,NY GiantsNYG,12.0,6.0,6.0,0.0,238.0,76.0,35.0,2.2,162.0,183.0,7.3,0.0,64.0,3.7,1978-09-02,new york giants vs tampa bay buccaneers,NaN,NaN,25.0,12.0,1.0,3.0,46.0,0.0,0.0,2.0,2.0,5.0,18.0,27.0,9.0,43.3,7.0,64.0,4.0,21.0,4.0,11.0,4.0,103.0,27.0,NYG,Tampa BayTB,16.0,9.0,4.0,3.0,251.0,165.0,39.0,4.2,86.0,93.0,3.3,0.0,68.0,3.7,1978-09-02,new york giants vs tampa bay buccaneers,NaN,NaN,28.0,10.0,3.0,1.0,3.0,4.0,1.0,2.0,2.0,4.0,17.0,23.0,7.0,44.3,8.0,55.0,1.0,7.0,5.0,76.0,5.0,86.0,23.0,TB,1978-09-02 NYG vs TB
1,Green BayGB,16.0,11.0,2.0,3.0,212.0,181.0,55.0,3.3,31.0,31.0,3.4,0.0,64.0,3.3,1978-09-03,green bay packers vs detroit lions,NaN,NaN,9.0,3.0,0.0,1.0,10.0,1.0,0.0,2.0,2.0,8.0,18.0,44.0,7.0,40.6,8.0,40.0,0.0,0.0,2.0,15.0,2.0,45.0,44.0,GB,DetroitDet,11.0,5.0,6.0,0.0,122.0,82.0,26.0,3.2,40.0,116.0,6.1,0.0,53.0,2.3,1978-09-03,green bay packers vs detroit lions,NaN,NaN,19.0,10.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,15.0,26.0,7.0,38.7,9.0,31.0,8.0,76.0,5.0,46.0,4.0,91.0,26.0,DET,1978-09-03 DET vs GB
2,HoustonHou,13.0,2.0,9.0,2.0,261.0,156.0,21.0,7.4,105.0,136.0,3.8,1.0,61.0,4.3,1978-09-03,houston oilers vs atlanta falcons,NaN,NaN,36.0,19.0,2.0,0.0,0.0,4.0,2.0,0.0,0.0,4.0,15.0,26.0,9.0,37.1,7.0,52.0,4.0,31.0,4.0,23.0,5.0,169.0,26.0,HOU,AtlantaAtl,11.0,6.0,4.0,1.0,171.0,100.0,35.0,2.9,71.0,104.0,4.5,0.0,62.0,2.8,1978-09-03,houston oilers vs atlanta falcons,NaN,NaN,23.0,10.0,0.0,2.0,14.0,3.0,1.0,2.0,3.0,2.0,14.0,14.0,9.0,42.7,8.0,75.0,4.0,33.0,6.0,59.0,3.0,80.0,14.0,ATL,1978-09-03 ATL vs HOU
3,Kansas CityKC,26.0,14.0,9.0,3.0,388.0,267.0,69.0,3.9,121.0,133.0,6.7,1.0,90.0,4.3,1978-09-03,kansas city chiefs vs cincinnati bengals,NaN,NaN,20.0,14.0,0.0,2.0,0.0,4.0,2.0,1.0,1.0,12.0,21.0,57.0,5.0,36.2,9.0,82.0,1.0,12.0,2.0,16.0,4.0,104.0,57.0,KC,CincinnatiCin,12.0,3.0,8.0,1.0,205.0,77.0,17.0,4.5,128.0,159.0,6.6,0.0,45.0,4.6,1978-09-03,kansas city chiefs vs cincinnati bengals,NaN,NaN,24.0,10.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,4.0,10.0,40.0,4.0,35.8,4.0,30.0,4.0,31.0,4.0,29.0,5.0,89.0,40.0,CIN,1978-09-03 CIN vs KC
4,Los AngelesLA,15.0,5.0,9.0,1.0,220.0,103.0,33.0,3.1,117.0,154.0,4.7,0.0,71.0,3.1,1978-09-03,los angeles rams vs philadelphia eagles,NaN,NaN,33.0,14.0,0.0,1.0,8.0,3.0,1.0,3.0,4.0,5.0,15.0,33.0,9.0,43.3,10.0,82.0,5.0,37.0,4.0,26.0,3.0,45.0,33.0,LA,PhiladelphiaPhi,10.0,5.0,4.0,1.0,208.0,128.0,30.0,4.3,80.0,102.0,6.0,1.0,50.0,4.2,1978-09-03,los angeles rams vs philadelphia eagles,NaN,NaN,17.0,7.0,1.0,0.0,0.0,2.0,1.0,0.0,1.0,3.0,13.0,23.0,8.0,36.8,3.0,38.0,3.0,22.0,7.0,76.0,2.0,48.0,23.0,PHI,1978-09-03 LA vs PHI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

Scraping each book individually is going to be a pain, let's use the odds api to get the odds for every game for every bookmaker this week

In [ ]:
with open('api-key.txt','r') as api_key_file:
    api_key = api_key_file.read()
    api_key_file.close()

odds_req = requests.get(f"https://api.the-odds-api.com//v4/sports/americanfootball_nfl/odds/?apiKey={api_key}&regions=us,eu&markets=h2h,totals,spreads")

In [ ]:

odds_json = json.loads(odds_req.content)
odds_df = pd.json_normalize(odds_json, record_path=['bookmakers','markets','outcomes'], meta=['id','commence_time','home_team','away_team',['bookmakers','title'],['bookmakers','markets','key'],])
odds_df

,name,price,point,id,commence_time,home_team,away_team,bookmakers.title,bookmakers.markets.key
0,Dallas Cowboys,1.22,NaN,b2eeb176fc9adfc63b9098b313905792,2023-12-01T01:15:00Z,Dallas Cowboys,Seattle Seahawks,FanDuel,h2h
1,Seattle Seahawks,4.50,NaN,b2eeb176fc9adfc63b9098b313905792,2023-12-01T01:15:00Z,Dallas Cowboys,Seattle Seahawks,FanDuel,h2h
2,Dallas Cowboys,1.91,-8.5,b2eeb176fc9adfc63b9098b313905792,2023-12-01T01:15:00Z,Dallas Cowboys,Seattle Seahawks,FanDuel,spreads
3,Seattle Seahawks,1.91,8.5,b2eeb176fc9adfc63b9098b313905792,2023-12-01T01:15:00Z,Dallas Cowboys,Seattle Seahawks,FanDuel,spreads
4,Over,1.93,47.5,b2eeb176fc9adfc63b9098b313905792,2023-12-01T01:15:00Z,Dallas Cowboys,Seattle Seahawks,FanDuel,totals
...,...,...,...,...,...,...,...,...,...
2875,Under,1.91,44.5,2c4ca7353e2db06d12d583f8a46d38a7,2023-12-12T01:16:00Z,Miami Dolphins,Tennessee Titans,Bovada,totals
2876,Miami Dolphins,1.91,-12.5,2c4ca7353e2db06d12d583f8a46d38a7,2023-12-12T01:16:00Z,Miami Dolphins,Tennessee Titans,BetUS,spreads
2877,Tennessee Titans,1.91,12.5,2c4ca7353e2db06d12d583f8a46d38a7,2023-12-12T01:16:00Z,Miami Dolphins,Tennessee Titans,BetUS,spreads
2878,Over,1.91,44.5,2c4ca7353e2db06d12d583f8a46d38a7,2023-12-12T01:16:00Z,Miami Dolphins,Tennessee Titans,BetUS,totals


# Exploratory Data Analysis

Let's look at how often teams actually cover the spread. In a perfect world (for the bookmakers), the probability of a team covering the spread would be 50/50. However, let's see the actual numbers

In [ ]:
games['spread_favorite']
games.dropna(subset = ['spread_favorite','team_favorite_id'],inplace=True)
def covered(row):
    if row['team_favorite_id'] == row['home_abrev']:
        point_diff = row['score_home']-row['score_away']
    else:
        point_diff = row['score_away'] - row['score_home']
    if point_diff > np.abs(row['spread_favorite']):
        return True
    elif point_diff < np.abs(row['spread_favorite']):
        return False
    else:
        return np.nan
games['favorite_covered'] = games.apply(covered,axis=1)
plt.bar(x=["favorite did not cover", "favorite covered"] ,height = games['favorite_covered'].value_counts())

It looks like the spread favorite only covers the spread about 5195 / (5195 + 5570) * 100 = 48.25824431% of the time! which means the books are approximately 1.75% off when calculating their spread probabilities. 
While this may look like a small percentage, it's enough to work with and give me hope that we can in fact beat the books.


Let's look at the relationship between the predicted ELO probabilities and the actual win rates of games

In [ ]:
plt.scatter(x=master_df['elo1_pre'],y=master_df['elo2_pre'],)

In [ ]:
games['total'] = games['score_away'] + games['score_home']
highest_scoring = games.nlargest(n=250,columns='total')
highest_scoring.describe()

Let's examine some of the highest ELO teams in the Superbowl era. By examining these teams we can try to look at similarities between them and see if we can identify common factors that indicate a team is on the rise. 

In [ ]:
# first lets find the stronger opponent heading into a given matchup
sb_era_elo.loc[:,'stronger_team'] = sb_era_elo.loc[:,['elo1_pre','elo2_pre']].max(axis=1).copy()
#Now lets find the 250 strongest rated teams of the superbowl era and look at some summary statistics
strongest_250 = sb_era_elo.nlargest(250, columns='stronger_team')
strongest_250

In [ ]:
display(strongest_250.columns)
def find_stronger_qb(row):
    if row['stronger_team'] == row['elo1_pre']:
        return row['qbelo1_pre']
    else:
        return row['qbelo2_pre']
strongest_250['stronger_team_qb_elo'] = strongest_250.apply(find_stronger_qb,axis=1)
strongest_250['stronger_team_qb_elo']


In [ ]:
plt.bar(x=["strongest 250 team qb elo", "average qb elo"],height = [strongest_250['stronger_team_qb_elo'].mean(),(sb_era_elo['qbelo1_pre'].mean() + sb_era_elo['qbelo2_pre'].mean())/2])

These stats can give us a good idea of what an elite NFL team looks like. 

We can check out the correlation matrix for the superbowl era to give us a good idea of which variables are strongly related to each other

In [ ]:
sb_era_elo.corr(numeric_only=True)

#### Hypothesis
Games played in colder weather will tend to be lower scoring and therefore will not hit the over

In [ ]:
#Let's test out our theory
freezing = games[games['weather_temperature'] <= 32]
freezing['over'].value_counts()

It doesn't look like it makes a difference, however there might be another explanation...

In [ ]:
display(games['over_under_line'].describe())
display(freezing['over_under_line'].describe())

It looks like bookmakers are already adjusting for the weather. We need to start thinking outside the box and look for factors they haven't though about yet if we want to get an edge. 

Lets look at games that hit the over vs games that did not hit the over

In [ ]:

games['over'] = games['over'].map({0:'Red', 1:'Green', 2:'Blue'})
games.plot.scatter(x='over_under_line',y='total',c='over', alpha = .3)

This scatter plot shows us games that hit the over in green, games that hit the under in red, and games that pushed in blue, with total points on the y-axis, and the over under line on the x. This is a good way to visualize the data as we can clearly see where the over-under line is on the plot. 

# Building a Model

Onto the good stuff:
My approach for my model is will be:
- Build a Machine Learning model using the master dataframe that predicts the probability of a team winning a given game (classification)
- Build a second model that predicts point totals of a game (regression)
- potential independent variables for the models will be, Team ELOs, QB Elos, sportsbook odds, season record, momentum score (fraction of x previous games won), weather, injuries, and any other useful statitistics I can find
- Dependent variables for the model will be the winner/win probability for the first model, and the predicted score for each team in the second model
- Test the model using cross validation
- Use the predictions and probabilities from the model, along with the new odds from the Odds API to identify potential positive EV bets
- Test to see if our identified "positive EV" bets are actually profitable. 
- Repeat until we make a model that is profitable

The First Model! Let's start with a logistic regression model to predict scores for each team

In [ ]:
master_df.columns

In [ ]:
master_df[['elo1_pre','elo2_pre','elo_prob1','elo_prob2','qbelo1_pre','qbelo2_pre','quality','importance','over_under_line','spread_favorite','weather_temperature']]

In [ ]:
x_train_dict = master_df[['elo1_pre','elo2_pre','elo_prob1','elo_prob2','qbelo1_pre','qbelo2_pre','quality','importance','over_under_line','spread_favorite',]].dropna(axis=1).to_dict(orient='records')
y_train = master_df['score1']
vec = DictVectorizer(sparse = False)
vec.fit(x_train_dict)
x_train = vec.transform(x_train_dict)
scaler = StandardScaler()
scaler.fit(x_train)
x_train_sc = scaler.transform(x_train)
model = LogisticRegression(solver='newton-cg',max_iter=8000)
model.fit(x_train_sc,y_train)
#let's test on our training data 
y_pred = model.predict(x_train_sc)
y_pred
mean_absolute_error(master_df['score1'],y_pred)

Yikes, that's a big error, and that's only our TRAINING error. We have some work to do...

We need to make sure we are getting the best odds from books, so let's write a function that gives us the best odds in odd_df

In [ ]:
def find_best_odds(odds_df):
    '''
    function that performs a line search on the odds dataframe to find and return the best odds for each game
    returns: list of dataframes of the best odds for each market of each game
    '''
    ids = pd.unique(odds_df['id'])
    markets = pd.unique(odds_df['bookmakers.markets.key'])
    games = odds_df.groupby(by=['id','bookmakers.markets.key'])
    all_games_lst = []
    best_bets = []
    for id in ids:
        for market in markets:
            if market in odds_df[odds_df['id'] == id]['bookmakers.markets.key'].tolist():
                game_market_odds = odds_df.iloc[games.groups[(id,market)]]
                all_games_lst.append(game_market_odds)
    for game in all_games_lst:
        h2h_odds = game[game['bookmakers.markets.key'] == 'h2h']
        h2h_lay_odds = game[game['bookmakers.markets.key'] == 'h2h_lay']
        spread_odds = game[game['bookmakers.markets.key'] == 'spread']
        over_under_odds = game[game['bookmakers.markets.key'] == 'totals']

        home_odds_h2h = h2h_odds[h2h_odds['name'] == h2h_odds['home_team']]
        away_odds_h2h = h2h_odds[h2h_odds['name'] == h2h_odds['away_team']]
        best_home_odds_h2h = home_odds_h2h.loc[home_odds_h2h['price'] == home_odds_h2h['price'].max()]
        best_bets.append(best_home_odds_h2h)
        best_away_odds = away_odds_h2h.loc[away_odds_h2h['price'] == away_odds_h2h['price'].max()]
        best_bets.append(best_away_odds)


        home_odds_h2h_lay = h2h_lay_odds[h2h_lay_odds['name'] == h2h_lay_odds['home_team']]
        away_odds_h2h_lay = h2h_lay_odds[h2h_lay_odds['name'] == h2h_lay_odds['away_team']]
        best_home_odds_h2h_lay = home_odds_h2h_lay.loc[home_odds_h2h_lay['price'] == home_odds_h2h_lay['price'].max()]
        best_bets.append(best_home_odds_h2h_lay)
        best_away_odds_h2h_lay = away_odds_h2h_lay.loc[away_odds_h2h_lay['price'] == away_odds_h2h_lay['price'].max()]
        best_bets.append(best_away_odds_h2h_lay)

        home_odds_spread = spread_odds[spread_odds['name'] == spread_odds['home_team']]
        away_odds_spread = spread_odds[spread_odds['name'] == spread_odds['away_team']]
        spreads = pd.unique(spread_odds['point'])
        for spread in spreads:
            home_line_odds_spread = home_odds_spread.loc[home_odds_spread['price'] == spread]
            best_home_odds_for_spread = home_line_odds_spread.loc[home_line_odds_spread['price'] == home_line_odds_spread['price'].max()]
            best_bets.append(best_home_odds_for_spread)

            away_line_odds_spread = away_odds_spread.loc[away_odds_spread['price'] == spread]
            best_away_odds_for_spread = away_line_odds_spread.loc[away_line_odds_spread['price'] == away_line_odds_spread['price'].max()]
            best_bets.append(best_away_odds_for_spread)
     
        
        home_odds_over_under = over_under_odds[over_under_odds['name'] == over_under_odds['home_team']]
        away_odds_over_under = over_under_odds[over_under_odds['name'] == over_under_odds['away_team']]
        lines = pd.unique(over_under_odds['point'])
        for line in lines:
            home_line_odds = home_odds_over_under.loc[home_odds_over_under['price'] == line]
            best_home_line_price = home_line_odds.loc[home_odds_over_under['price'] == home_odds_over_under['price'].max()]
            best_bets.append(best_home_line_price)

            away_line_odds = away_odds_over_under.loc[away_odds_over_under['price'] == line]
            best_away_line_price = away_line_odds.loc[away_odds_over_under['price'] == away_odds_over_under['price'].max()]
            best_bets.append(best_away_line_price)

    return best_bets
    
best = find_best_odds(odds_df)

#remove empty dfs from best
best_odds = []
for df in best:
    if not df.empty:
        best_odds.append(df)
best_odds

[               name  price  point                                id  \
 82   Dallas Cowboys   1.25    NaN  b2eeb176fc9adfc63b9098b313905792   
 102  Dallas Cowboys   1.25    NaN  b2eeb176fc9adfc63b9098b313905792   
 120  Dallas Cowboys   1.25    NaN  b2eeb176fc9adfc63b9098b313905792   
 140  Dallas Cowboys   1.25    NaN  b2eeb176fc9adfc63b9098b313905792   
 
             commence_time       home_team         away_team bookmakers.title  \
 82   2023-12-01T01:15:00Z  Dallas Cowboys  Seattle Seahawks        Matchbook   
 102  2023-12-01T01:15:00Z  Dallas Cowboys  Seattle Seahawks           Unibet   
 120  2023-12-01T01:15:00Z  Dallas Cowboys  Seattle Seahawks         Pinnacle   
 140  2023-12-01T01:15:00Z  Dallas Cowboys  Seattle Seahawks          Betfair   
 
     bookmakers.markets.key  
 82                     h2h  
 102                    h2h  
 120                    h2h  
 140                    h2h  ,
                  name  price  point                                id  \
 141  

Now that we have the best possible odds, we can calculate our expected value of our bet

First use Kelly Criterion to size our bet:

f<sup>*</sup> = p - (1-p)/b 


**f<sup>*</sup>** is the fraction of our bankroll we should put on the bet

**p** is our estimated probability of winning

**b** is the proportion of the bet we stand to win (eg for 2:1 odds b =2)

In [ ]:
def size_kelly_bet(bankroll, win_prob,odds):
    return bankroll * (win_prob - (1-win_prob)/odds)

Next let's calculate our expected value

In [ ]:
def calc_ev(bet_size,odds,win_prob):
    ev = bet_size*odds*win_prob - bet_size(1-win_prob)
    return ev

and now let's search for positive EV bets!

In [ ]:
#Final function should look something like this:
def find_plus_ev(win_prob,bankroll):
    plus_ev = []
    for game in best_odds:
        # win_prob = model.predict(game[cols])
        odds = game['price']
        bet_size = size_kelly_bet(bankroll,win_prob,odds)
        ev = calc_ev(bet_size,odds,win_prob)
        if ev > 0:
            plus_ev.append({'team':game['name'],'sportsbook':game['bookmakers.title'],'bet-size':bet_size,'odds':odds,'point':game['point'],'market':game['bookmakers.markets.key']})
    return plus_ev
